<h2 style = "text-align:center" ><b>EEG</b> - Prédiction des Mouvements Imaginaires de la Main</h2>

---

#### **1. Le projet**
- Intoduction  
https://github.com/DataScientest-Studio/mar24_cds_eeg/blob/eric/references/Description_projet_EEG.pdf  
https://www.bbci.de/competition/iv/desc_2b.pdf
- Ressources / Données   
https://www.kaggle.com/competitions/ucsd-neural-data-challenge/overview  
- Bibliographie  
https://www.bbci.de/competition/iv/desc_2b.pdf
#### **2. Liens utils**
- SciPy - *open-source software for mathematics, science, and engineering*  
https://docs.scipy.org/doc/scipy/index.html  
https://docs.scipy.org/doc/scipy/reference/signal.html  
- MNE - *MEG + EEG Analysis & Visualisation*
   - Accueil  
   https://mne.tools/stable/index.html

   - MNE - Data structures from arbitrary data  
   https://mne.tools/stable/auto_tutorials/io/10_reading_meg_data.html#creating-mne-data-structures-from-arbitrary-data-from-memory
   
   - MNE - EEG Preprocessing  
   https://mne.tools/dev/auto_tutorials/preprocessing/index.html  

- pyRiemann - *Biosignals classification with Riemannian geometry*  
https://pyriemann.readthedocs.io/en/latest/  
- neurodsp - *Neuro Digital Signal Processing Toolbox*  
https://neurodsp-tools.github.io/neurodsp/index.html#
- Rythme Mu  
https://fr.wikipedia.org/wiki/Rythme_Mu
- Spectrogram from EEG  
https://www.kaggle.com/code/cdeotte/how-to-make-spectrogram-from-eeg
- Divers  
https://signalprocessingsociety.org/  
https://fr.wikipedia.org/wiki/Filtre_de_Butterworth  
https://fr.wikipedia.org/wiki/Moyenne_mobile  
https://terpconnect.umd.edu/~toh/spectrum/Differentiation.html  
https://perso.etis-lab.fr/ghaffari/2014_CCMB_Floride_USA.pdf  
https://www.youtube.com/watch?v=wB417SAbdak&list=PLXc9qfVbMMN2TAoLHVW5NvNmJtwiHurzw  
https://fastercapital.com/fr/sujet/identification-des-artefacts-de-traitement-du-signal-dans-des-sc%C3%A9narios-r%C3%A9els.html#:~:text=L'inspection%20visuelle%20est%20la,des%20pertes%20et%20du%20bruit.  
   - Z-Score Normalisation  
   https://fr.wikipedia.org/wiki/Cote_Z_(statistiques)  
   https://typeset.io/questions/why-is-z-score-normalisation-necessary-in-pre-processing-eeg-1xv5jepyq5  

   - Traitement numérique du signal  
   https://fr.wikipedia.org/wiki/Traitement_num%C3%A9rique_du_signal  
   - Ondelette  
      - Wiki  
      https://fr.wikipedia.org/wiki/Ondelette  

      - L’analyse par ondelettes dans la vie de tous les jours  
      https://interstices.info/lanalyse-par-ondelettes-dans-la-vie-de-tous-les-jours/  

      - A guide for using the Wavelet Transform in Machine Learning  
      https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/
      
      - pyWavelets - *open source wavelet transform*  
      https://pywavelets.readthedocs.io/en/latest/

      - Ondelettes et applications  
      https://www.i2m.univ-amu.fr/~caroline.chaux/GEOMDATA/TI-te5215.pdf

   - Maximum de vraisemblance  
   https://pmarchand1.github.io/ECL8202/notes_cours/03-Maximum_vraisemblance.html  
   https://fr.wikipedia.org/wiki/Maximum_de_vraisemblance#:~:text=En%20statistique%2C%20l'estimateur%20du,maximisant%20la%20fonction%20de%20vraisemblance  

   - Transformation de Fourier discrète  
   https://fr.wikipedia.org/wiki/Transformation_de_Fourier_discr%C3%A8te  
      - La Transformation de Fourier n’est pas adaptée à l’analyse des signaux non stationnaires.
   - Neural Data Science in Python  
   https://neuraldatascience.io/intro.html

   - Preprocessing of EEG  
   https://www.frontiersin.org/articles/10.3389/fninf.2015.00016/full#:~:text=The%20depositable%20preprocessing%20pipeline%20consists,with%20a%20low%20recording%20SNR  
   https://typeset.io/papers/preprocessing-of-eeg-4go8vhcbty  
   https://learn.neurotechedu.com/preprocessing  
   https://g0rella.github.io/gorella_mwn/preprocessing_eeg.html  
   
   - Biblio :  
   https://perso.telecom-paristech.fr/bloch/P6Image/ondelettestrsp.pdf  
   https://www.math.u-bordeaux.fr/~jbigot/Site/Enseignement_files/ondelettesIMAT.pdf  
   http://w3.cran.univ-lorraine.fr/perso/radu.ranta/pdf/cours_deb_ond%28fr%29.pdf
   
   - Digital Filtering  
   http://notebooks.pluxbiosignals.com/notebooks/Categories/Pre-Process/digital_filtering_eeg_rev.html

   - Processus stationnaire  
   https://fr.wikipedia.org/wiki/Processus_stationnaire

   - Analyse en composantes principales  
   https://fr.wikipedia.org/wiki/Analyse_en_composantes_principales#:~:text=L'ACP%2C%20d%C3%A9sign%C3%A9e%20en%20g%C3%A9n%C3%A9ral,une%20grandeur%20physique%2C%20comme%20les

#### • Chargement des différentes librairies

In [2]:
### • Chargement des différentes librairies

import sys, os, gc

sys.path.insert(0, os.path.dirname(os.getcwd()))

from src.thot.sesh import *

from sklearn import model_selection
from sklearn import ensemble, svm, neighbors
# from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from keras.models import Sequential                             # type: ignore
from keras.callbacks import EarlyStopping                       # type: ignore
from keras.layers import GlobalAveragePooling1D                 # type: ignore
from keras.layers import Dense, Dropout, Conv1D, LSTM           # type: ignore
from keras.layers import LeakyReLU, ReLU, PReLU, ConvLSTM1D     # type: ignore
# from keras.layers import Bidirectional, TimeDistributed, RepeatVector, Flatten

# import tensorflow as tf
# from tensorflow.keras.optimizers import AdamW, Adam            # type: ignore

# from pyriemann.spatialfilters import CSP

import pywt, librosa
import seaborn as sns

# from scipy.fft import fft

%matplotlib inline

#### • Déclaration de constantes

In [ ]:
### • Déclaration de constantes

# Fréquence d'échantillonnage - Hz (Nombre de valeur / sec)
SAMPLE_RATE  = 250
# Temps additionel pour étendre le domaines d'étude.
LAG : int    = -64     # Décalage du signal dû signal acoustique ~500ms
#
PW2 : int    = int(np.floor(np.log2(SAMPLE_RATE)))
#
NFFT : int   = 1 << PW2
# Epoque en sec donnée en nombre d'échantillon consectutif # 4" de données (multiple de 2)
SCOPE : int  = (1 << PW2) << 2
# Deux enregistrements bipolaires + neutre
eeg_Chans    = ['C3', 'C4', 'Cz']
# Liste des cannaux eeg associés aux évènement 0 et 1
eeg_left     = [f'{c}_0' for c in eeg_Chans]
eeg_right    = [f'{c}_1' for c in eeg_Chans]
full_eeg     = eeg_left + eeg_right
# Trois enregistrements musculaires
ecg_Chans    = ['EOG:ch01', 'EOG:ch02', 'EOG:ch03']
# Liste de tous les cannaux des dataframes
all_chans    = eeg_Chans + ecg_Chans
# Correspondance pour la classification
hands_event  = {0: 'Left', 1: 'Right'}
# Les bandes de fréquences d'intérêt
eeg_bands    = {'Delta' : (0.1, 4.0),
                'Theta' : (4.1, 8.0),
                'Alpha' : (8.1, 14.0),
                'Beta'  : (14.1, 30.0),
                'Gamma' : (30.1, (SAMPLE_RATE >> 1) - 1),}
# Coefficients pour filtres Butterworth numérique d'ordre N pour le filtrage passe-bande
bands_coeff  = {band : butter_bandpass(low, high, SAMPLE_RATE) for band, (low, high) in eeg_bands.items()}
# Nombre dévènement à prédir
num_events   = range(len(hands_event))

LAG, SCOPE

#### • Acquisition des données d'entrainement

In [3]:
### • Acquisition des données d'entrainement

target  = "../data/data.zip"
count   = len('train/')
fics    = [x[count :] for x in files_in_zip(target, directory = 'train')]

[fics.remove(x) for x in fics[:: -3]]

# Acquisition des fichiers du répertoir dans le fichier zip
train_csv = csv_in_zip(target, directory = 'train', files = fics)
label_csv = csv_in_zip(target, directory = 'y_train_only', files = fics)

notes   = filename(fics)
headers = [f"{t} . {i + 1}" for i, t in enumerate(notes)]
count   = range(len(train_csv))

# fics    = [f'B0{i}0{j}T.csv' for i in range(1, 9) for j in range(1, 4)]
# df_train_pkl = pkl_in_zip(path, fichier_specifique = 'epoched_train.pkl')

#### • Acquisition des données de test

In [ ]:
### • Acquisition des données de test

test_csv = csv_in_zip(target, directory = 'test')

#### • Pré-traitement des données

In [12]:
### • Pré-traitement des données

def data_spliting(datas : list[Board], Channels : Clause, number_event : int | Index,
                  labels : list[Board] | None = None, merge : bool = False, level : bool = True) :
    count = range(len(datas))
    parts = []        #
    temp  = [[], []]  # Les époques pour tous les cannaux et tous les évènements.
    spots = [[], []]  # Apparitions des évènements
    # Pour la standardisation du nombre d'échantillon max conservé
    ceil  = min([len(labels[i]['EventType']) for i in count])

    if type(number_event) == int : number_event = range(number_event)

    # Extraction des données relavitives à l'apparition des évènements.
    if level :
        for i in count :
            df   = datas[i]
            kind = labels[i]['EventType'][: ceil]
            loc  = np.where(df['EventStart'] == 1)[: ceil]

            parts.append(zero_removal(df['C3'], 75))

            for j in number_event :
                spots[j].append(np.array(*loc)[*np.where(kind == j)])

                # [tf.convert_to_tensor(X) for X in event_epochs(spots[j][-1], SCOPE, LAG)]
                room = event_epochs(spots[j][-1], SCOPE, LAG)

                temp[j].append([full_event(df[c], room, merge) for c in Channels])
    else :
        for i in count :
            df   = datas[i]
            kind = label_csv[i]['EventType']
            loc  = np.where(df['EventStart'] == 1)

            parts.append(zero_removal(df['C3'], 75))

            for j in number_event :
                spots[j].append(np.array(*loc)[*np.where(kind == j)])

                room = event_epochs(spots[j][-1], SCOPE, LAG)

                temp[j].append([full_event(df[c], room, merge) for c in Channels])

    # Regroupement des données en fonction du type de l'évènement et du cannal d'observation
    if merge :
        temp = [[[np.append([], T[j :: 3]) for T in temp[i]] for j in range(3)] for i in number_event]
    else :
        store = [[[], [], []], [[], [], []]]
        
        [[[[store[i][j].append(G) for G in X] for j, X in enumerate(T)] for T in temp[i]] for i in number_event]

        temp = store

    eras = [pd.DataFrame({**dict(zip(Channels, [pd.Series(X) for X in temp[i]])), 'EventType': i}) for i in number_event]

    return eras, spots, parts

train_eras, train_spots, train_parts = data_spliting(train_csv, eeg_Chans, num_events, label_csv)

train_samples = samples(len(train_eras[0]))
trains        = pd.concat(train_eras, ignore_index = True)

#### • Test prédiction

In [255]:
### • Visualisation des spectrogrammes / Test

def logMelSpectrogram(data : Vector, rate : int, dt : float = 1e-2) -> Vector :
    tps = 1 << int(np.floor(np.log2(rate * dt)))
    # print(tps)
    # Spectrogramme
    stfts = np.abs(librosa.stft(y = data, n_fft = tps, hop_length = 1 << 2, center = True)).T
    # Filtre de MEL
    liny  = librosa.filters.mel(sr = rate, n_fft = tps + 1, n_mels = stfts.shape[-1]).T
    # Application du filtre au spectrogramme
    mel_  = np.tensordot(stfts, liny, 1)

    return np.log(mel_ + 1e-6)
    
def structure(data : Board | Vector, rate : int, whr : Clause) -> Vector :
    # return np.array([logMelSpectrogram(X, rate, 2) for X in data[whr]])
    # return np.stack([[signal.welch(X, rate)[1] for X in data[c]] for c in whr], axis = 2)
    return np.stack(trains['C4'] - trains['C3'], axis = 1)
    # return np.stack([[X for X in data[c]] for c in whr], axis = 2)

def img_spectrogram(raw : Vector, rate : int, nfft : int = 1 << 10) -> Vector :
    return librosa.feature.melspectrogram(y = raw, sr = rate, hop_length = 1, 
                            n_fft = nfft, n_mels = 32, fmin = 0, fmax = 20, win_length = 32)

def spectrogram(data : Board, rate : int, channels : Clause, n_row : int = 5, n_col : int = 12) :
    sample = np.random.default_rng().integers(data.shape[0], size = n_row)

    sample.sort()

    plt.figure(figsize = (18, 2 * .48 * n_row))

    pos = 0

    for k in sample :
        for c in channels :
            x   = normalized(data[c][k])
            raw = img_spectrogram(raw = x, rate = rate)
            pos += 1
            
            plt.subplot(n_row, n_col, pos)
            plt.title(f"{((pos - 1) // 3) + 1} . {k} - {c}", fontsize = 8)
            librosa.display.specshow(data = 1 - raw, sr = rate, hop_length = 1)
            
            # pos = n_col * (i >> 1) + j
            # f, t, Sxx = signal.spectrogram(x, rate)
            # plt.subplot(n_row, n_col, pos + 4)
            # plt.pcolormesh(t, f, 1 - Sxx, shading = 'gouraud')

    plt.tight_layout()
    plt.show();

def plot_logMelSpectrogram(data, rate) :
    sns.heatmap(np.rot90(logMelSpectrogram(data, rate)), cmap = 'inferno')
    
    # loc, _ = plt.xticks()
    # l      = np.round((loc - loc.min()) * len(data) / fe / loc.max(), 2), vmin = -6

    # plt.xticks(loc, l)
    plt.yticks([])
    plt.xlabel("Time (s)")
    plt.ylabel("Frequency (Mel)")

In [243]:
H = [list(harmonic(x, bands_coeff).values()) for x in [trains[c] for c in eeg_Chans]]

In [ ]:
np.shape(H), np.shape(np.stack(np.stack(H, axis = 1), axis = 2)), np.shape(H[0][1][0])
# harmonic(trains['C3'][256], bands_coeff).values())), H

In [250]:
X_train, X_test, y_train, y_test = train_test_split(trains.drop(columns = ['EventType']),
                                                    trains['EventType'], test_size = .2, random_state = 42)

# X_train, X_test, y_train, y_test = train_test_split([v.tolist() for v in np.array(trains[eeg_Channels])],
#                                                     trains['EventType'], test_size = .2, random_state = 42)

In [256]:
# train_dataset = np.array(X_train)
# test_dataset  = X_test

# train_dataset = structure(X_train, SAMPLE_RATE, eeg_Chans[:2])
# test_dataset  = structure(X_test, SAMPLE_RATE, eeg_Chans[:2])

train_dataset = structure(X_train, SAMPLE_RATE, eeg_Chans[:2])
test_dataset  = structure(X_test, SAMPLE_RATE, eeg_Chans[:2])

# print(train_dataset.shape)

In [ ]:
np.array(X_train).shape

In [ ]:
#### • Test prédiction

# UNITS     : int   = 100
BATCHSIZE : int   = 30
EPOCH     : int   = 1000
ZERO      : int   = 64
DROPOUT   : float = .2
# kl_divergence mean_squared_logarithmic_error mean_absolute_error
LOSS      : None  = 'sparse_categorical_crossentropy'
ACTIV     : None  = LeakyReLU   # PReLU, 
OPTIMIZER : None  = 'AdamW'     # adamax, , adafactor, adam, nadam

model = Sequential()

# - 1 -
model.add(Conv1D(filters = ZERO, kernel_size = (5), dilation_rate = 2,
                 input_shape = train_dataset.shape[1: ]))
model.add(Dropout(DROPOUT))
model.add(ACTIV())

# - 2 -
model.add(Conv1D(filters = ZERO << 1, kernel_size = (5), dilation_rate = 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())

# - 3 -
model.add(Conv1D(filters = ZERO << 2, kernel_size = (5), dilation_rate = 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())
model.add(GlobalAveragePooling1D()) # 

# Classification
model.add(Dense(ZERO << 2))
model.add(Dropout(DROPOUT))
model.add(ACTIV())
model.add(Dense(len(hands_event), activation = 'softmax')) # sigmoid  

'''
# Ajout de la premiere couche lstm
model.add(LSTM(ZERO, input_shape = train_dataset.shape[1:], activation = ACTIV(), return_sequences = True)) #
model.add(LSTM(ZERO, dropout = DROPOUT, return_sequences = False))

# Ajout de la couche de sortie
model.add(Dense(len(hands_event), activation = 'softmax'))
'''

model.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = ['accuracy'])
# model.summary();

In [ ]:
stop    = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 50)
history = model.fit(train_dataset, y_train, validation_data = (test_dataset, y_test), verbose = 1,
                    batch_size = BATCHSIZE, epochs = EPOCH, callbacks = [stop])

In [ ]:
pred = model.predict(test_dataset)

sum([np.where(x > .5)[0][0] for x in pred] == y_test) / len(pred)

In [ ]:
history_dict = history.history
loss_values  = history_dict['loss']
acc_values   = history_dict['accuracy']
absc         = range(1, len(loss_values) + 1)

plt.figure(figsize = (12, 4))

plt.subplot(121)
plt.plot(absc, loss_values, label = 'Loss')
plt.plot(absc, acc_values, label = 'Accuracy')
plt.title('Training')
plt.xlabel('Epochs')
plt.legend()

plt.subplot(122)
plt.plot(absc, history_dict['val_loss'], label = 'Loss')
plt.plot(absc, history_dict['val_accuracy'], label = 'Accuracy')
plt.title('Testing')
plt.xlabel('Epochs')
plt.legend()
plt.show();

#### • Visualisation densité spectrale du Signal

In [ ]:
### • Visualisation Densité Spectral du Signal

plot_psd(train_csv, train_eras, rate = SAMPLE_RATE, Channels = eeg_Chans, titled = headers)

#### • Densité spectrale / échantillon

In [ ]:
### • Densité spectral / échantillon

n   = 40
scp = samples(train_samples, n)
pos = -2

plt.figure(figsize = (15, n * 1.5))

for i in scp :
    pos += 2

    for c in eeg_Chans :
        x = train_eras[0][c][i]
        f, Pxx_den = signal.welch(x, SAMPLE_RATE)   # , scaling = 'spectrum'
        
        plt.subplot(n, 4, pos + 1)
        plt.semilogy(f, Pxx_den, label = c)
        plt.title(f"welch - {i + 1}", fontsize = 11)
        plt.grid()

        plt.subplot(n, 4, pos + 2)
        r, _ = plt.psd(x, Fs = SAMPLE_RATE, label = c) # , NFFT = NFFT
        plt.title(f"psd - {i + 1}", fontsize = 11)
        plt.xlabel('')
        plt.ylabel('')
        # plt.legend()

plt.xlabel('frequency [Hz]')
# plt.ylabel('PSD [V**2/Hz]')
plt.tight_layout()
plt.show();

# f, Pxx_den = signal.welch(train_eras[0]['C3'][752], SAMPLE_RATE)

# print(len(Pxx_den))

#### • Visualisation Epoques

In [ ]:
### • Visualisation Epoques

for i in range(len(fics))[:: 3] :
    plot_signal(train_csv[i], train_parts[i], train_spots[0][i], train_spots[1][i], channels = eeg_Chans, # 
                period = SCOPE, lag = LAG, title = headers[i])

#### • Visualisation décomposition des signaux

In [ ]:
### • Visualisation Décomposition des signaux

# Test de décomposition des signaux en bandes de fréquences spécifiques compatibles avec les répartitions usuelles
# dans le domaine des EEG ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']

for df, token in zip(train_eras, ['Gauche', 'Droite']) :
    print(f"Exemples - Évènement Discriminé Main {token}")
    plot_wavelets(df, bands_coeff, eeg_Chans, scope = 30, headers = headers)

#### • Visualisation des spectrogrammes (test)

In [ ]:
for i, t in zip(num_events, ['Gauche', 'Droite']) :
    print(f"Exemples - Évènement Discriminé Main {t}")
    spectrogram(train_eras[i], SAMPLE_RATE, eeg_Chans, 160)

#### • Test PCA - (Non cloncluant)

In [ ]:
### • Test PCA - (Non cloncluant)

nca = SCOPE >> 2
pca = PCA(nca)

_, ax = plt.subplots(nrows = 2, ncols = 3, figsize = (15, 5))

for i, d in enumerate(train_eras) :
    for j, c in enumerate(eeg_Chans) :
        Z = sc.fit_transform(list(d[c].to_list())) # 
        principal_components = pca.fit_transform(Z)
        
        ax[i, j].plot(range(nca), np.cumsum(pca.explained_variance_ratio_))
        ax[i, j].set_title(f'{c} . {i}')

plt.tight_layout()
plt.show();

#### • MNE époque (test)

In [ ]:
### • MNE époque (test)

raw_csv = train_csv[0][eeg_Chans]
info    = mne.create_info(ch_names = eeg_Chans, sfreq = SAMPLE_RATE, ch_types = 'eeg')
raw_mne = mne.io.RawArray(raw_csv.T * 1e-6, info)
loc     = np.where(train_csv[0]['EventStart'] == 1)[0]

# display(compare(np.sort(np.concatenate((train_spots[0][0], train_spots[1][0]))), loc))

tmin, tmax = -0., 1

# loc = mne.find_events(raw_mne, stim_channel = 'C3')
# event_id = dict(C3 = 1, aud_r = 2, vis_l = 3, vis_r = 4)
# raw = mne.io.Raw(raw_mne, preload = True)
# raw.filter(2, None, method = 'iir')           # replace baselining with high-pass
# events = mne.read_events(event_fname)

# raw.info['bads'] = ['MEG 2443']  # set bad channels
# picks = mne.pick_types(info, meg = 'grad', eeg = True, eog = False, exclude = 'bads')
# Read epochs
absc = mne.Epochs(raw_mne, np.array([loc, loc, loc]).T, None, tmin, tmax, proj = False,
                    picks = None, baseline = None, preload = True, verbose = False) # event_id picks

# labels = epochs.events[::5, -1]

# events

# raw_mne.plot();

# raw_mne['C3'][0][0], len(df_train_csv[2]['Cz'])

display(absc)

#### • Test de classification - Proposition inputs 01

In [17]:
#### • Test de classification - Proposition inputs 01

# X1 = np.where(trains['EventType'] == 0, trains['C3'], trains['Cz']) 
# X2 = np.where(trains['EventType'] == 0, trains['Cz'], trains['C4']) 
# X = [signal.welch(list(v), SAMPLE_RATE)[1] for v in X1 + X2]

# X = [np.append([], list(harmonic(v, bands_coeff).values())) for v in X]
# X = np.where(trains['EventType'] == 0, trains['C3'], 2 * trains['Cz'])
# X = [signal.welch(list(v), SAMPLE_RATE)[1] for v in X], trains['C3'] + trains['Cz'], trains['C4'] + trains['Cz']

diff = np.array([trains['C3'] - trains['C4']])
X = [np.append([], v.tolist()) for v in diff.T]

# X = [np.append([], v.tolist()) for v in np.array(trains[eeg_Chans])]
# diff = 
# X = np.where(trains['EventType'] == 0, trains['C3'], 2 * trains['Cz'])[:2]
# X = [np.append([], list(harmonic(v, bands_coeff).values())) for v in X]
# # X = [list(v) for v in X]

# pca = PCA()
# X = pca.fit_transform(X)
# X = [signal.welch(v, SAMPLE_RATE)[1] for v in X]

In [ ]:
print(diff.T.shape), np.array(trains[eeg_Chans]).shape

#### • Test de classification - Proposition inputs 02

In [ ]:
### • Test de classification

X_train, X_test, y_train, y_test = train_test_split(X, trains['EventType'],
                                                    test_size = .2, random_state = 42)

sc = StandardScaler()

X_train_scaled = sc.fit_transform(X_train)
X_test_scaled  = sc.transform(X_test)

# X_train_scaled = X_train
# X_test_scaled  = X_test

# 
csp = CSP(nfilter = 2)
# K-plus proches voisins
knc = neighbors.KNeighborsClassifier()
# SVM (support vector machine)[, 'auto', kernel = 'rbf']
clf = svm.SVC(gamma = 'scale')
# RandomForest 
rfc = ensemble.RandomForestClassifier(n_jobs = -1)
#
lrg = LogisticRegression()
#ExtraTreesClassifier 
# Voting_clf = VotingClassifier(estimators = [('knn', clf1), ('svm', clf2), ('rf', clf3)], voting = 'hard')
# cv3        = model_selection.KFold(n_splits = 3, random_state = 42, shuffle = True), clf4

# Create a pipeline
pip = Pipeline([('CSP', csp), ('RFC', rfc), ('SVM', clf)])

# pip.fit(X_train, y_train)

# y_pred = pip.predict(X_test)

y_train = np.array(y_train)

# print("Classification report:\n", classification_report(y_test, y_pred))
# print("Accuracy score:", accuracy_score(y_test, y_pred)), lrg

for clf in [knc, clf, rfc, csp] :
    print(f"● {clf} :")

    scores : dict = model_selection.cross_validate(clf, X_train, y_train, cv = 5, scoring = ['accuracy'])
    
    r = scores['test_accuracy']
    
    print(f"  - Accuracy : {r.mean():.1%} (±{r.std(): .2})")
    print()

In [ ]:
svm.fit(X_train_scaled, y_train)

# print('score train :', clf3.score(X_train_scaled, y_train))
print('score test :', svm.score(X_test_scaled, y_test))

pred = svm.predict(X_test_scaled)

In [ ]:
params = {
    ## K-plus proches voisins
    'knn__n_neighbors' : range(2),
    ## SVM
    'svm__C'      : [0.1, 1, 5],
    'svm__kernel' : ['linear', 'softmax', 'sigmoid', 'rbf'],
    ## RandomForest
    # 'rf__max_features'      : ['sqrt', 'log2', None],
    # 'rf__min_samples_split' : range(2, 32, 2),
    # , ('rf', clf3), ('rf', clf3)
    'estimators': [[('knn', knc), ('svm', svm)], [('knn', knc), ('svm', svm)]] 
    }

grid = model_selection.GridSearchCV(estimator = Voting_clf, param_grid = params, cv = 5) \
    .fit(X_train_scaled, y_train)

# parametres = {'max_features' : ['log2', 'sqrt', None], 'min_samples_split' : range(2, 32, 2)}

# vclf = model_selection.GridSearchCV(estimator = clf3, param_grid = parametres, cv = 3) \
#     .fit(X_train_scaled, y_train)

print(grid.best_estimator_)
print(grid.best_score_)
print('score train :', grid.score(X_train_scaled, y_train))
print('score test :', grid.score(X_test_scaled, y_test))

# print(vclf.best_estimator_, vclf.best_params_, vclf.best_score_)
# print('score train :', grid.score(X_train_scaled, y_train), vclf.score(X_train_scaled, y_train))
# print('score test  :', grid.score(X_test_scaled, y_test), vclf.score(X_test_scaled, y_test))

df_train_cpy, event_start = fancy_df(train_csv, label_csv['EventType'], hands_event, SCOPE)

fig, axes = plt.subplots(1, 1, figsize = (24, 5), sharey = True)
sig = .05

axes.plot(train_csv['C3'])

for p in event_start :
    axes.axvspan(p[0] - (SCOPE >> 1), p[0] + 1.5 * SCOPE, facecolor = 'orangered', alpha = .5)

plt.xlabel('Frequency (Hz)')
plt.ylabel('Score')

In [ ]:
pos    = 16
start  = event_start[pos][0]
entrant  = start + SCOPE
df   = df_train_cpy['C3_4'][start : entrant]
smooth = df.copy()
n      = 5
alpha  = 1 / 3
dec    = int(n / alpha)

plt.figure(figsize = (24, 5))
plot_window(train_csv, ['C3', 'C4', 'C3 + C4'], start, SCOPE)

# Lissage des hautes fréquences
for _ in range(n) :
    smooth = simple_exponential_smoothing(smooth, alpha, 0)

smooth = pd.Series(index = range(start, entrant + n - dec), data = smooth[dec :])

# plt.plot(raw - smooth, label = hands[event_start[pos][1]])
plt.plot(smooth, '--', label = hands_event[event_start[pos][1]])
plt.legend()
plt.show()

In [ ]:
'''
- Introduction :  
L’électroencéphalogramme (EEG) est une technique d’imagerie cérébrale utilisée pour étudier les activités du cerveau.  
En plaçant des capteurs sur le cuir chevelu, l’activité électrique du cerveau est enregistrée,  
   ce qui permet de comprendre les fonctionnements cérébraux et d’identifier certains schémas que l’on peut ensuite attribuer à des comportements précis.  
Un des schémas d'EEG qui a été beaucoup étudié est l’imagerie motrice (IM), ou le mouvement imaginaire de la main.  
Les IM créent des schémas bien définis qui peuvent être détectés.  
Le but de ce projet est de créer et d’entraîner un programme permettant de prédire si l’IM d’une personne correspond à un mouvement de la main droite ou de la main gauche.  
# **2. Étapes du projet**
- Prétraitement des Données :  
Les données EEG sont sujettes à des artefacts ou des erreurs de collecte dues à des mouvements parasites ou des interférences.  
Il est donc nécessaire d'appliquer un système de prétraitement des données pour réduire le bruit et extraire les bandes de fréquences pertinentes.

- Segmentation des données et extraction des caractéristiques :  
Les données EEG sont présentées comme un flux continu. Il est donc important, pour une meilleure analyse, de diviser les données en segments temporels correspondant à l’IM.  
Ensuite, identifier et extraire les caractéristiques pertinentes des signaux EEG associées aux IM est essentiel.  
Cela comprend la puissance et d'autres spécificités de l’activité électrique qui définissent les IM.

- Analyse statistique exploratoire :  
Utiliser les outils d’analyse exploratoire pour mieux comprendre les données et identifier les tendances ou les patterns significatifs.

- Entraînement du modèle :  
Entraîner un modèle permettant de distinguer les différences entre les IM des mains droite et gauche.  
Optimiser le modèle et évaluer sa performance sur un ensemble de test.

- Conclusion :  
Ces étapes sont cruciales pour développer un programme efficace de prédiction des mouvements imaginaires de la main basé sur les données EEG.
'''

'''
parts = []        #
temp = [[], []]  # Les époques pour tous les cannaux et tous les évènements.
spots = [[], []]  # Apparitions des évènements

# Pour la standardisation du nombre d'échantillon max conservé
ceil   = min([len(label_csv[i]['EventType']) for i in count])
merge  = False

# Extraction des données relavitives à l'apparition des évènements.
for i in count :
    df   = train_csv[i]
    kind  = label_csv[i]['EventType'][: ceil]
    loc = np.where(df['EventStart'] == 1)[: ceil]

    parts.append(zero_removal(df['C3'], 75))

    for i in num_events :
        spots[i].append(np.array(*loc)[*np.where(kind == i)])

        room = event_epochs(spots[i][-1], SCOPE, LAG)

        temp[i].append([full_event(df[c], room, merge)for c in eeg_Channels])

# Regroupement des données en fonction du type de l'évènement et du cannal d'observation
if merge :
    temp = [[[np.append([], T[j :: 3]) for T in temp[i]] for j in range(3)] for i in num_events]
else :
    store = [[[], [], []], [[], [], []]]
    
    [[[[store[i][j].append(G) for G in X] for j, X in enumerate(T)] for T in temp[i]] for i in num_events]

    temp = store
    # res = [[np.concatenate(np.stack(tries[i], axis = 1)[j], axis = 0) for j in range(3)] for i in n_type]

eras    = [pd.DataFrame(dict(zip(eeg_Channels, [pd.Series(X) for X in temp[i]]))) for i in num_events]
'''